## Duplicate memory.

In this notebook, we will test that our ``agent`` is capable of handling duplicate memories.
Through its ``memory manager``, that will identify if a new memory is a duplicate by checking similarity based on context and tokens.
If a duplicate memory is found, the policy is to ignore the older ``memory`` in favor of the newer ``memory``.

In [1]:
from goodai.src.models import OpenSourceModel, Agent

In [2]:
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"

In [3]:
model = OpenSourceModel(model_name)
agent = Agent(model)

[2024-03-01 21:43:22,647]:[root]:[INFO]: connection to session database established.
[2024-03-01 21:43:22,649]:[sentence_transformers.SentenceTransformer]:[INFO]: Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


First let's start with a clean ``session``, a session is a combinations of interactions with the ``agent``, these interactions are saved in a local ``sqlite`` database in order to preserve them, if we start a new session all information provided to the ``agent`` will be lost.

In [4]:
agent.new_session()

[2024-03-01 21:43:29,809]:[root]:[INFO]: Cleared session database.


Now, let's provide our ``agent`` with some informations.

In [5]:
agent.interact("I am going outside.")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

'Great, have a nice time outside! Let me know if you need anything.'

*Agent*: 'Great, have a nice time outside! Let me know if you need anything.'

In [6]:
agent.interact("I have returned home and am going to take a bath.")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

"Got it! Enjoy your bath. I'll be here if you need anything."

*Agent*: "Got it! Enjoy your bath. I'll be here if you need anything."

Now, let's check the content of our memory buffer.

In [8]:
agent.memory_manager.memory_buffer

[Memory(content=I am going outside., memory_type=new, timestamp=2024-03-01 21:43:22.232147, expiration=2024-04-30 21:43:22.232147),
 Memory(content=I have returned home and am going to take a bath., memory_type=new, timestamp=2024-03-01 21:43:22.232147, expiration=2024-04-30 21:43:22.232147)]

This is the content of our memory buffer:

- Memory(content=I am going outside., memory_type=new, timestamp=2024-03-01 21:34:02.652218, expiration=2024-04-30 21:34:02.652218),
- Memory(content=I have returned home and am going to take a bath., memory_type=new, timestamp=2024-03-01 21:34:02.652218, expiration=2024-04-30 21:34:02.652218)

Now let's provide the same input to the ``agent``

In [9]:
agent.interact("I am going outside again.")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[2024-03-01 21:44:25,991]:[root]:[WARNING]: Duplicated memory found.


'Alright, have a great time outside! Let me know if you need anything.'

*Agent*: 'Alright, have a great time outside! Let me know if you need anything.'

But the ``memory manager`` of the ``agent`` was able to detect that the last interaction is in fact a duplicate of a prior interaction, so the older memory is removed and the newer memory is inserted in its buffer, we can confirm this by checking its content.

In [10]:
agent.memory_manager.memory_buffer

[Memory(content=I have returned home and am going to take a bath., memory_type=new, timestamp=2024-03-01 21:43:22.232147, expiration=2024-04-30 21:43:22.232147),
 Memory(content=I am going outside again., memory_type=new, timestamp=2024-03-01 21:43:22.232147, expiration=2024-04-30 21:43:22.232147)]